This code shows how to use vertices information to find keypoints using the data from the simulation itself.

In this work we assume that the vertices are saved in npy files and that per every frame we have the location and orientation and scale of the mesh.

####NOTE
The vertices in blender of the fbx file might be different than the vertices used in the USD file (ordering).
You might need to check this.
In our case what we did was processing every asset in omniverse and export the .npy containing all the vertices information (see the main GRADE repository on how to do that).

After that the npy can be loaded in blender with 

```
import bpy
import numpy as np


context = bpy.context
load it into another array
verts = np.load('/ps/project/irotate/syn_zebras/Zebra_anims/Idle4_points.npy', allow_pickle=True)[0]

new mesh
me = bpy.data.meshes.new("Verts")
load in verts
me.from_pydata(verts, [], [])
create object and link to scene
vob = bpy.data.objects.new("Verts", me)
context.collection.objects.link(vob)
print()
bpy.ops.object.mode_set(mode='OBJECT')
selectedVerts = [v for v in bpy.context.active_object.data.vertices if v.select]
l = [v.index for v in selectedVerts]
print(l)
bpy.ops.object.mode_set(mode='EDIT')
```

And the vertices visualized

Pre-load the npy files with the points information

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import trimesh
import os
import scipy.spatial.transform as tf
import math 
import scipy 

# this is to get the vertices of the mesh
zebra_anims = {}
anim_dir = "/ps/project/irotate/syn_zebras/Zebra_anims"
for anim in os.listdir(anim_dir):
    if "_points.npy" in anim:
        zebra_anims[anim[:anim.rfind("_")]] = np.load(os.path.join(anim_dir,anim), allow_pickle=True)

Define the vertices that will consitute each keypoints. We'll average them

In [3]:

vertical_aperture = 2.3189

complete_joints = {
    #todo complete this
'left_back_paw' : [4456, 4485, 4490, 4504, 4544, 4547, 4548, 4561, 4593, 4608, 4610, 4621, 4646, 4656, 4674, 4691, 4696, 4725, 4781, 4861, 4867, 4875, 4880, 4903, 4925, 4953, 5021, 5050, 5053, 5059, 5066, 5078, 5230, 5240, 5242, 5247, 5248, 5262, 5275, 5298, 5373, 5380, 5387, 5403, 5441, 5444, 5445, 5452, 5478, 5485, 5488, 5510, 5567, 5569, 5585, 5586, 5599, 5602, 5629, 5653, 5684, 5709, 5715, 5716, 5729, 5733, 5743, 5786, 5787, 5791, 5802, 5803, 5810, 5821, 5836, 5840, 5857, 5882, 5911, 5922],
'left_back_knee' : [4441, 4443, 4474, 4494, 4501, 4505, 4506, 4536, 4540, 4558, 4600, 4609, 4625, 4631, 4634, 4639, 4649, 4670, 4731, 4744, 4774, 4779, 4814, 4820, 4846, 4855, 4906, 4911, 4912, 4932, 5052, 5062, 5074, 5085, 5125, 5139, 5146, 5161, 5167, 5206, 5343, 5348, 5351, 5354, 5366, 5367, 5374, 5377, 5395, 5456, 5466, 5475, 5489, 5507, 5516, 5527, 5570, 5590, 5601, 5612, 5628, 5652, 5666, 5672, 5680, 5687, 5705, 5708, 5713, 5717, 5721, 5722, 5724, 5736, 5745, 5746, 5749, 5756, 5777, 5789, 5794, 5796, 5799, 5804, 5808, 5834, 5844, 5847, 5850, 5851, 5854, 5855, 5859, 5862, 5866, 5871, 5874, 5879, 5893],
# 'left_back_thigh' : [4118, 4157, 4162, 4200, 4210, 4255, 4313, 4339, 4402, 4454, 4666, 4741, 4868, 4874, 5180, 5217, 5427, 5465, 5576, 5581, 5878, 5923, 5969, 5974, 6088, 6096, 6099, 6108, 6144, 6148, 6152, 6162, 6166, 6167, 6170, 6173, 6174, 6176, 6178, 6184],
'left_back_thigh' : [6149, 6155, 6158, 6159, 6171, 6172, 6179, 6180, 6181, 6182, 6183, 6185, 6186, 6187, 6188],

'right_back_paw' : [235, 236, 246, 256, 267, 278, 300, 307, 321, 349, 353, 364, 383, 386, 387, 419, 432, 446, 456, 473, 521, 543, 560, 563, 587, 596, 603, 604, 676, 690, 704, 711, 719, 747, 774, 785, 796, 809, 818, 915, 935, 943, 944, 949, 951, 960, 979, 1106, 1120, 1129, 1142, 1243, 1270, 1273, 1284, 1300, 1311, 1313, 1330, 1464, 1476, 1488, 1520, 1542, 1591, 1619, 1632, 1650, 1655, 1662, 1672, 1705, 1711, 1715, 1724, 1728, 1747, 1752, 1758],
'right_back_knee' : [296, 310, 315, 318, 323, 327, 330, 334, 335, 338, 339, 342, 345, 355, 381, 385, 390, 393, 395, 400, 412, 433, 440, 444, 453, 465, 472, 476, 481, 509, 517, 537, 561, 577, 599, 619, 682, 700, 714, 724, 795, 797, 813, 817, 825, 826, 837, 841, 987, 1027, 1048, 1057, 1111, 1122, 1131, 1143, 1286, 1292, 1358, 1365, 1392, 1397, 1430, 1438, 1547, 1578, 1580, 1588, 1603, 1614, 1653, 1671, 1673, 1703, 1704, 1708, 1713, 1734, 1768, 1769],
# 'right_back_thigh' : [15, 19, 21, 22, 23, 24, 25, 28, 37, 45, 50, 81, 93, 101, 131, 215, 220, 311, 389, 608, 613, 764, 777, 975, 1014, 1280, 1340, 1474, 1551, 1719, 1754, 1808, 1894, 1940, 1983, 2029, 2036, 2078],
'right_back_thigh' : [1, 2, 3, 4, 6, 7, 8, 9, 10, 17, 18, 30, 31, 34, 40, 82, 85, 116],

'right_front_paw' : [218, 231, 242, 243, 254, 265, 291, 293, 295, 326, 340, 341, 344, 354, 359, 401, 420, 427, 479, 571, 575, 583, 606, 616, 618, 623, 647, 688, 824, 838, 864, 904, 923, 958, 965, 971, 1105, 1134, 1186, 1187, 1197, 1278, 1312, 1354, 1391, 1394, 1417, 1420, 1454, 1463, 1535, 1539, 1545, 1558, 1559, 1567, 1611, 1612, 1613, 1627, 1635, 1638, 1643, 1645, 1659],
'right_front_knee': [269, 283, 304, 319, 337, 346, 363, 409, 428, 442, 459, 463, 487, 488, 499, 511, 598, 600, 665, 709, 748, 787, 789, 801, 957, 998, 1024, 1043, 1051, 1093, 1162, 1200, 1219, 1249, 1262, 1277, 1316, 1317, 1344, 1377, 1413, 1422],
# 'right_front_thihg' : [106, 123, 139, 162, 178, 181, 204, 213, 229, 263, 277, 392, 437, 642, 687, 691, 728, 919, 939, 968, 1042, 1116, 1227, 1306, 1414, 1415, 1468, 1494, 1508, 1543, 1649, 1692, 1772],
'right_front_thihg' : [47, 48, 57, 58, 67, 83, 86, 87, 98, 103, 132, 152, 175, 182, 196],

'left_front_paw' : [4551, 4566, 4570, 4576, 4580, 4586, 4601, 4602, 4604, 4644, 4651, 4653, 4671, 4677, 4680, 4750, 4762, 4788, 4790, 4818, 4821, 4860, 4894, 4919, 5001, 5008, 5010, 5058, 5093, 5221, 5226, 5232, 5270, 5290, 5321, 5328, 5353, 5369, 5501, 5542, 5566, 5571, 5573, 5583, 5604, 5606, 5614, 5618, 5710, 5762, 5769, 5788, 5830, 5835, 5845, 5848, 5849, 5863, 5894, 5896, 5898, 5924, 5935, 5946, 5947, 5958, 5971],
'left_front_knee': [4786, 4795, 4835, 4866, 4889, 4890, 4920, 4933, 4946, 4984, 4999, 5034, 5106, 5145, 5150, 5173, 5197, 5233, 5389, 5402, 5404, 5443, 5480, 5524, 5589, 5591, 5678, 5690, 5701, 5702, 5726, 5730, 5747, 5761, 5780, 5826, 5843, 5852, 5870, 5885, 5906, 5920],
# 'left_front_thigh' : [4438, 4514, 4562, 4673, 4701, 4718, 4746, 4792, 4793, 4898, 4972, 5152, 5224, 5253, 5271, 5498, 5502, 5547, 5559, 5752, 5797, 5926, 5960, 5976, 5985, 6008, 6011, 6027, 6050, 6066, 6083],
'left_front_thigh' : [5692, 5779, 5837, 5944, 5993, 6004, 6007, 6037, 6049, 6057, 6091, 6097, 6122, 6131, 6142],
    
'tail_end' : [2552, 2564, 2573, 2627, 2694, 2724, 2726, 2728, 2748, 2771, 2777, 2809, 2822, 2831, 2839, 2847, 2851, 2853, 2857, 2860, 2867, 2868, 2885, 2893, 2897, 2902, 2908, 2919, 2923, 2931, 2932, 2934, 2937, 2949, 2952, 2960, 2961, 2964, 2972, 2977, 2979, 2985, 2988, 2990, 2991, 2992, 2998, 3004, 3005, 3008, 3011, 3015, 3025, 3027, 3034, 3036, 3038, 3061, 3062, 3086, 3092, 3153, 3154, 3155, 3156, 3158, 3159, 3204, 3207, 3211, 3216, 3217, 3218, 3219, 3220, 3222, 3224, 3227, 3228, 3229, 3231, 3234, 3239, 3240, 3246, 3253, 3255, 3257, 3258, 3266, 3269, 3280, 3283, 3285, 3286, 3291, 3295, 3302, 3308, 3314, 3322, 3329, 3343, 3344, 3352, 3354, 3360, 3363, 3365, 3373, 3381, 3389, 3400, 3435, 3439, 3470, 3493, 3496, 3500, 3532, 3601, 3652, 3658, 3668],
'tail_base' : [2121, 2194, 2214, 2244, 2316, 2332, 2334, 2342, 2359, 2369, 2430, 2444, 2450, 2506, 2548, 2594, 2603, 2647, 2660, 2760, 2779, 2793, 2815, 2869, 2873, 2898, 2924, 2956, 2959, 3100, 3101, 3102, 3107, 3259, 3262, 3290, 3313, 3340, 3342, 3393, 3417, 3430, 3451, 3562, 3578, 3629, 3670, 3712, 3781, 3854, 3862, 3874, 3880, 3882, 3901, 3979, 4004, 4017],

'right_ear_tip' : [783, 803, 819, 827, 847, 855, 869, 874, 887, 895, 924, 962, 966, 986, 1022, 1036, 1047, 1078, 1084, 1099, 1123, 1139, 1153, 1168, 1179, 1189, 1190, 1217, 1258, 1315, 1320],
'right_ear_base' : [976, 990, 1089, 1154, 1328, 1341, 1584, 1585, 1726, 1738, 1748, 1786, 1834, 1842],

'left_ear_tip' : [4825, 4830, 4881, 4928, 4948, 4950, 4963, 4969, 4988, 5005, 5024, 5041, 5047, 5049, 5089, 5099, 5114, 5155, 5174, 5178, 5209, 5252, 5254, 5281, 5286, 5299, 5309, 5324, 5334, 5349, 5368],
'left_ear_base' : [4343, 4347, 4397, 4436, 4445, 4458, 4589, 4594, 4842],

'right_eye' : [1261, 1264, 1272, 1274, 1283, 1287, 1296, 1297, 1298, 1301, 1303, 1307, 1329, 1335, 1339, 1343, 1345, 1351, 1368, 1370, 1373, 1378, 1390, 1400, 1407, 1409, 1410, 1419, 1427, 1429, 1431, 1433, 1436, 1440, 1443, 1446, 1448, 1449, 1458, 1465, 1471, 1473, 1477, 1479, 1484, 1485, 1489, 1491, 1510, 1513, 1515, 1516, 1522, 1524, 1529, 1530, 1532, 1536, 1540, 1548, 1550, 1553, 1561, 1562, 1570, 1572, 1574, 1577, 1579, 1586, 1589, 1593, 1599, 1600, 1602, 1606, 1608, 1616, 1620, 1623, 1624, 1626, 1629, 1631, 1633, 1639, 1657, 1665, 1667, 1674, 1686, 1691, 1694, 1696, 1697, 1699, 1707, 1712, 1714, 1716, 1718, 1721, 1729, 1735, 1736, 1741, 1745, 1750, 1759, 1762, 1763, 1764, 1765, 1770, 1771, 1773, 1775, 1776, 1780, 1782, 1783, 1787, 1789, 1795, 1800, 1803, 1807, 1809, 1815, 1817, 1820, 1821, 1823, 1827, 1841, 1847, 1848, 1851, 1852, 1856, 1858, 1863, 1866, 1868],
'left_eye' : [4267, 4269, 4275, 4280, 4281, 4282, 4293, 4297, 4304, 4309, 4312, 4319, 4320, 4325, 4326, 4333, 4334, 4336, 4344, 4350, 4353, 4357, 4358, 4361, 4363, 4368, 4375, 4378, 4381, 4383, 4387, 4393, 4396, 4398, 4399, 4405, 4409, 4411, 4412, 4414, 4422, 4423, 4424, 4428, 4432, 4434, 4439, 4440, 4447, 4448, 4450, 4455, 4462, 4464, 4465, 4468, 4473, 4476, 4478, 4481, 4484, 4492, 4493, 4503, 4507, 4512, 4515, 4518, 4524, 4525, 4528, 4537, 4538, 4542, 4543, 4545, 4552, 4553, 4564, 4565, 4569, 4572, 4573, 4575, 4584, 4585, 4591, 4597, 4599, 4603, 4605, 4611, 4614, 4617, 4622, 4623, 4624, 4632, 4633, 4637, 4638, 4641, 4647, 4652, 4655, 4659, 4661, 4662, 4663, 4665, 4668, 4669, 4675, 4683, 4688, 4692, 4697, 4700, 4703, 4709, 4711, 4719, 4724, 4727, 4728, 4732, 4735, 4738, 4742, 4752, 4760, 4763, 4765, 4769, 4773, 4776, 4780, 4783, 4797, 4800, 4803, 4815, 4833, 4834, 4837, 4841, 4849, 4853],

'nose' : [1885, 1934, 1948, 1969, 1990, 1998, 2021, 2051, 2060, 2080, 2086, 2117, 2130, 2142, 2157, 2178, 2181, 2182, 2191, 2193, 2203, 2205, 2206, 2212, 2218, 2220, 2222, 2226, 2231, 2235, 2237, 2243, 2246, 2249, 2250, 2252, 2253, 2254, 2260, 2261, 2262, 2264, 2268, 2269, 2270, 2271, 2272, 2273, 2274, 2277, 2278, 2279, 2281, 2283, 2285, 2286, 2289, 2290, 2291, 2299, 2300, 2306, 2307, 2315, 3892, 3898, 3899, 3902, 3905, 3906, 3908, 3910, 3912, 3913, 3914, 3915, 3917, 3918, 3919, 3920, 3921, 3922, 3923, 3924, 3926, 3934, 3936, 3939, 3941, 3944, 3946, 3947, 3950, 3954, 3959, 3965, 3968, 3972, 3976, 3980, 3981, 3983, 3985, 3986, 3997, 4012, 4013, 4014, 4021, 4031, 4049, 4063, 4071, 4077, 4091, 4097, 4107, 4122, 4186, 4239],

'neck_start' : [524, 657, 743, 856, 862, 1087, 1220, 1497, 1523, 1877, 1908, 2134, 2161, 2493, 2527, 2731, 2774, 3055, 3166, 3431, 3490, 3685, 3725, 4045, 4066, 4295, 4331, 4687, 4715, 4981, 5112, 5330, 5336, 5448, 5532, 5665],
'neck_end' : [956, 1010, 1059, 1065, 1098, 1114, 1541, 1640, 1959, 2287, 2557, 3176, 3486, 3665, 3951, 4235, 4577, 4689, 5091, 5102, 5130, 5136, 5185, 5238, 2738],
'skull': [2385, 2390, 2392, 2403, 2619, 2628, 2632, 2638, 2687, 2999, 3000, 3001, 3002, 3003, 3456, 3515, 3530, 3531, 3539, 3761, 3778],

'body_middle' : [62, 63, 68, 69, 88, 89, 111, 119, 137, 138, 144, 154, 287, 294, 297, 301, 652, 664, 686, 702, 1086, 1107, 1201, 1356, 1648, 1656, 1792, 1831, 2008, 2009, 2083, 2111, 2412, 2434, 2514, 2531, 2699, 2722, 2773, 2805, 3111, 3112, 3113, 3118, 3402, 3437, 3501, 3525, 3682, 3705, 3791, 3813, 4088, 4112, 4184, 4185, 4377, 4419, 4555, 4563, 4858, 4998, 5090, 5111, 5487, 5503, 5525, 5537, 5888, 5892, 5895, 5902, 6035, 6045, 6051, 6052, 6070, 6078, 6100, 6101, 6120, 6121, 6126, 6127],
'back_end' : [30, 34, 40, 65, 117, 194, 426, 720, 1173, 1661, 1985, 2344, 2643, 3133, 3582, 3873, 4208, 4549, 5028, 5469, 5763, 5995, 6072, 6124, 6149, 6155, 6159],
'back_front':[74, 77, 95, 103, 132, 152, 185, 260, 371, 485, 628, 880, 1405, 1837, 2136, 2485, 2702, 3165, 3523, 3734, 4067, 4372, 4805, 5313, 5561, 5704, 5818, 5929, 6004, 6037, 6057, 6086, 6094, 6112, 6115],
}

In [4]:
def project_im_to_world(points, horiz_fov, vert_fov, distance, c_view):
    """
    Project 2D points in image space to 3D points in world using a pinhole camera model.

    Args:
        points (numpy.ndarray): Array of points in world frame of shape (num_points, 3).
        viewport (omni.kit.viewport_legacy._viewport.IViewportWindow): Viewport from which to retrieve/create sensor.

    Returns:
        (numpy.ndarray): Image-space points of shape (num_points, 3)
    """
    radius = distance
    theta = horiz_fov * points[:, 0] / 2
    phi = vert_fov * points[:, 1] / 2

    # Convert from polar to cartesian
    y = radius * np.sin(phi)
    h = radius * np.cos(phi)
    x = h * np.sin(theta)
    z = -h * np.cos(theta)  # -Z is camera forward

    position_cam = np.stack([x, y, z], axis=1)
    position_world = np.pad(position_cam, ((0, 0), (0, 1)), constant_values=1) @ c_view
    return position_cam #position_world[:, :3]


def project_pinhole(points, view_proj_matrix, pose):
    """
    Project 3D points to 2D camera view using a pinhole camera model.
    Args:
      points (numpy.ndarray): Array of points in world frame of shape (num_points, 3).
      viewport (omni.kit.viewport._viewport.IViewportWindow): Viewport from which to retrieve/create sensor.

    Returns:
      (numpy.ndarray): Image-space points of shape (num_points, 3)
    """
    homo = np.pad(points, ((0, 0), (0, 1)), constant_values=1.0)
    tf_points_old = np.dot(homo, view_proj_matrix)
    tf_points = tf_points_old / (tf_points_old[..., -1:])
    tf_points[..., :2] = 0.5 * (tf_points[..., :2] + 1)
    return tf_points[..., :3], tf_points_old[...,3]

def centroid(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return sum_x/length, sum_y/length, sum_z/length

def test_visibility(coord, segm, depth_test, zebra_id, is_leg=False):
    """
    Test the visibility of the keypoint
    """
    shape = segm.shape
    if coord[1] > shape[0] or coord[0] > shape[1] or coord[0] < 0 or coord[1] < 0:
        return 0
    if coord[1] == shape[0]:
        coord[1] -= 1
    if coord[0] == shape[1]:
        coord[0] -= 1
    if segm[coord[1],coord[0]] != zebra_id:
        return 0
    if depth_test:
        return not is_leg
    return 2


The main path is the folder with all the generation experiments.
This code will loop through the three viewports Viewport\[1,2,3\] and process segmentation depth and camera information.

With frame_info we load the information about the location of the zebra.

We then project the points from the npy files (corresponding to the mesh) and reproject back the centroids computed using the vertices information.

Easy enough and we got the keypoints.

In [11]:
main_path = '/ps/project/irotate/syn_zebras/zebra_close/'

exps = os.listdir(main_path)
for exp in exps:
    folder = os.path.join(main_path, exp) # exp folder
    print(folder)
    if '.' in exp or not os.path.isdir(folder):
        continue
    for fname in os.listdir(folder):
        if 'frame' not in fname and '.npy' not in fname:
            continue
        
        # load the necessary infomation. The fnumber is the image number, frame_info is the corresponding file containing
        # the information about the locations of the zebras, their scale, and their orientation
        fnumber = int(fname[fname.find("_")+1:fname.rfind("_")])*3 + int(fname[fname.rfind("_")+1:-4]) # get the frame id
        frame_info = np.load(os.path.join(folder,fname),allow_pickle=True).item()

        for viewport in ['Viewport0','Viewport1','Viewport2']:
            segmentation_info = np.load(os.path.join(folder, viewport, f'instance/{fnumber}.npy'), allow_pickle=True) # segm
            depth = np.load(os.path.join(folder,  viewport, f'depthLinear/{fnumber}.npy'), allow_pickle=True) # depth
            cam_info = np.load(os.path.join(folder,  viewport, f'camera/{fnumber}.npy'), allow_pickle = True).item() # camera

            # get unique ids from segmentation[0] matrix
            ids = np.unique(segmentation_info[0])
            if len(np.where(ids == 0)[0]) >= 0:
                ids = np.delete(ids, np.where(ids == 0)[0])
            ids -= 1 # offset back 1so that we can avoid the loop and directly work with indices in segmentation_info[1].
            
            # get only the visible zebras, i.e. the ones that have at least one pixel in the image
            visibles = [x for x in segmentation_info[1][ids] if "zebra" in x[1]]

            # create the output dir within the experiment
            output_dir = os.path.join(folder, viewport, 'keypoints')
            os.makedirs(output_dir, exist_ok=True)

            # todo put a for here to cycle through Viewports and frames
            im_kps = {}
            
            # visualization
#             import cv2 
#             rgb_data = cv2.imread(os.path.join(folder, viewport, f"rgb/{fnumber}.png"))
#             from PIL import Image, ImageDraw
#             rgb_img = Image.fromarray(rgb_data).convert("RGBA")
#             master_overlay_img = Image.new("RGBA", (1920, 1080), (0, 0, 0, 255))

            for vis in visibles:
                zebra = vis[1]
                if zebra not in frame_info:
                    # sometimes some error can happen due to zebras which are supposedly out of the frame but visible with just 1 pixels because in the "not visible location"
                    # you want to check this manually
                    print(f'ERROR WITH {os.path.join(folder,  viewport, f"camera/{fnumber}")} and zebra {zebra}')
                    continue

                # get all the info for the zebra
                z_animation = frame_info[zebra]['name']
                z_frame = frame_info[zebra]['used_frame']-1
                z_scale = frame_info[zebra]['scale'] 
                z_pose = frame_info[zebra]['position'] 
                z_orient = frame_info[zebra]['rotation']
                z_orient = tf.Rotation.from_euler("XYZ", z_orient).as_matrix()
                # scale the mesh
                curr_mesh = zebra_anims[z_animation][z_frame] * z_scale / 100

                # get the cam pose
                c_pose = cam_info["pose"]
                # get the view projection matrix
                c_view = cam_info["view_projection_matrix"]
                # get the camera matrix
                c_mat = np.dot(c_pose, c_view)
                # change the fov
                vfov =  2 * math.atan(vertical_aperture / (2 * cam_info["focal_length"]))
                c_mat[1,1] = 1 / math.tan(vfov / 2)
                # get the correct view projection matrix
                c_view = np.dot(np.linalg.inv(c_pose), c_mat)

                # put the mesh in the correct 3D world location
                tf_mesh = curr_mesh @ z_orient.T + z_pose
                # scale the mesh according to the scale of the simulation 
                tf_mesh = tf_mesh * 100
                all_points = tf_mesh

                # project it to the image plane, pending image size scaling
                projected_points = project_pinhole(tf_mesh, c_view, c_pose)

                zebra_instance_id = vis[0]

                # get the gt joint info
                zebra_kps = {}
                for joint, ids in complete_joints.items():
                    coords = [centroid(all_points[ids])]
                    img_coord, depth_scale = project_pinhole(coords, c_view, c_pose)
                    inverse_point = np.append(img_coord,1)
                    
                    img_coord = img_coord[...,:2].reshape(-1,2)*np.array([[1920, 1080]])
                    img_coord = img_coord.astype(int)
                    
                    # compute distance from camera pose of all vertices
                    dist = scipy.spatial.distance.cdist(all_points[ids],[c_pose[3,:3]])
                    
                    inverse_point[...,:2] = (2 * inverse_point[...,:2] - 1)
                    inverse_point = inverse_point * depth_scale
                    
                    depth_test = min(dist)[0] > np.linalg.norm(inverse_point.dot(np.linalg.inv(c_view))[:3]-c_pose[3,:3])
                    
                    is_leg = "paw" in joint or "knee" in joint
                    # assign the visibility value
                    vis = test_visibility(img_coord[0], segmentation_info[0], depth_test, zebra_instance_id, is_leg)
                    zebra_kps[joint] = [img_coord[0][0], img_coord[0][1], vis]

                im_kps[zebra[1:]] = zebra_kps
                
                # visualize
                # get the pixel coordinates
#                 rounded_points = projected_points.astype(int)

                # get the vertices that are visible
#                 values_at_coords = segmentation_info[0][rounded_points[:,1],rounded_points[:,0]]
#                 rounded_points = projected_points[values_at_coords == zebra_instance_id]


#                 overlay = Image.new("RGBA",(1920,1080))
#                 draw = ImageDraw.Draw(overlay)

#                 color = tuple(np.random.randint(size=3, low=0, high=256))
#                 for p in rounded_points:
#                     draw.point((p[0],p[1]),fill=color)
#                 draw.point((img_coord[0][0], img_coord[0][1]), fill="yellow")
#                 master_overlay_img = Image.alpha_composite(master_overlay_img, overlay)
#             plt.imshow(master_overlay_img)
#             import time
#             time.sleep(5)
            
#                 rgb_img = Image.alpha_composite(rgb_img, master_overlay_img)
#                 plt.imshow(master_overlay_img)
#                 master_overlay_img.save('/home/ebonetto/vertices_white-new.png')

            # todo fix this
            np.save(os.path.join(output_dir,f'{fnumber}.npy'), im_kps)
#             if viewport == 'Viewport2':
#                 break
#         break
#     break
    print("Ended")

/ps/project/irotate/syn_zebras/zebra_close/Forest
Ended
/ps/project/irotate/syn_zebras/zebra_close/Savana
Ended
/ps/project/irotate/syn_zebras/zebra_close/Windmills
Ended
/ps/project/irotate/syn_zebras/zebra_close/Woodland
ERROR WITH /ps/project/irotate/syn_zebras/zebra_close/Woodland/Viewport2/camera/302 and zebra /zebra_246
ERROR WITH /ps/project/irotate/syn_zebras/zebra_close/Woodland/Viewport2/camera/482 and zebra /zebra_13
Ended
/ps/project/irotate/syn_zebras/zebra_close/Iceland
Ended
/ps/project/irotate/syn_zebras/zebra_close/L_Terrain
Ended
/ps/project/irotate/syn_zebras/zebra_close/Grasslands
Ended
/ps/project/irotate/syn_zebras/zebra_close/Meadow
Ended
/ps/project/irotate/syn_zebras/zebra_close/Bliss
Ended
/ps/project/irotate/syn_zebras/zebra_close/Moorlands
Ended


In [9]:
vis = visibles[-3]
viewport = 'Viewport1'

In [10]:
import cv2 
rgb_data = cv2.imread(os.path.join(folder, viewport, f"rgb/{fnumber}.png"))
from PIL import Image, ImageDraw
rgb_img = Image.fromarray(rgb_data).convert("RGBA")
master_overlay_img = Image.new("RGBA", (1920, 1080), (0, 0, 0, 255))

zebra = vis[1]
z_animation = frame_info[zebra]['name']
z_frame = frame_info[zebra]['used_frame']-1
z_scale = frame_info[zebra]['scale'] 
z_pose = frame_info[zebra]['position'] 
z_orient = frame_info[zebra]['rotation']
z_orient = tf.Rotation.from_euler("XYZ", z_orient).as_matrix()
# scale the mesh
curr_mesh = zebra_anims[z_animation][z_frame] * z_scale / 100

# get the cam pose
c_pose = cam_info["pose"]
# get the view projection matrix
c_view = cam_info["view_projection_matrix"]
# get the camera matrix
c_mat = np.dot(c_pose, c_view)
# change the fov
vfov =  2 * math.atan(vertical_aperture / (2 * cam_info["focal_length"]))
c_mat[1,1] = 1 / math.tan(vfov / 2)
# get the correct view projection matrix
c_view = np.dot(np.linalg.inv(c_pose), c_mat)

# put the mesh in the correct 3D world location
tf_mesh = curr_mesh @ z_orient.T + z_pose
# scale the mesh according to the scale of the simulation 
tf_mesh = tf_mesh * 100
all_points = tf_mesh

# project it to the image plane, pending image size scaling
projected_points, _  = project_pinhole(tf_mesh, c_view)

# be sure that the points are within the FOV

#     mask_uv = ~np.any(np.all(projected_points < 0, axis=1), axis=0) & ~np.any(
#         np.all(projected_points > 1, axis=1), axis=0
#       )
#     mask_z = np.all(np.all(projected_points[..., 2:] >= 0, axis=1), axis=0) & np.all(
#         np.all(projected_points[..., 2:] <= 1, axis=1), axis=0
#       )
#     projected_points = projected_points[mask_uv & mask_z]
# new code
mask = np.all(projected_points <= 1, axis=1) & np.all(projected_points >= 0, axis=1)
projected_points = projected_points[mask]

# scaled based on the image size
projected_points = projected_points[...,:2].reshape(-1,2) * np.array([[1920, 1080]])


zebra_instance_id = vis[0]

# get the gt joint info
zebra_kps = {}
for joint, ids in complete_joints.items():
    coords = [centroid(all_points[ids])]
    img_coord, _ = project_pinhole(coords, c_view)
    img_coord = img_coord[...,:2].reshape(-1,2)*np.array([[1920, 1080]])
    img_coord = img_coord.astype(int)
    vis = test_visibility(img_coord[0], segmentation_info[0], depth, zebra_instance_id)
    zebra_kps[joint] = [img_coord[0][0], img_coord[0][1], vis]

# im_kps[zebra] = zebra_kps

# visualize
# get the pixel coordinates
rounded_points = projected_points.astype(int)

# get the vertices that are visible
values_at_coords = segmentation_info[0][rounded_points[:,1],rounded_points[:,0]]
rounded_points = projected_points[values_at_coords == zebra_instance_id]


overlay = Image.new("RGBA",(1920,1080))
draw = ImageDraw.Draw(overlay)

color = tuple(np.random.randint(size=3, low=0, high=256))
for p in rounded_points:
    draw.point((p[0],p[1]),fill=color)
#                 draw.point((img_coord[0][0], img_coord[0][1]), fill="yellow")
master_overlay_img = Image.alpha_composite(master_overlay_img, overlay)

In [11]:
plt.imshow(master_overlay_img)